# Tutorial 4: Calibration of a point SUMMA simulation with Ostrich

This notebook outlines the initial work on interfacing Ostrich into the `pysumma` workflow. 
The basic workflow is similar to that of the other `pysumma` objects, with some initialization of an object, configuration, and finally a `run` call.
Before digging in I'll describe at a high level detail how the `Ostrich` object works.
The `Ostrich` object basically manages the initial state of the calibration and will write out all of the necessary files to actually run the calibration.
It only is a very thin wrapper around a subprocess call out to `Ostrich` itself.
The `Ostrich` program will begin by setting initial parameter values and runs SUMMA through `run_script.py`, which is written by the `Ostrich` object.
This run script will also handle the updating of the SUMMA parameter datastructures and evaluating the run with several metrics which can be used as loss functions.
So far I have implemented KGE, MAE, and RMSE as well as a number of other helper pieces that I'll explain later.

As usual we will begin with some standard imports and configuration, along with loading in some data that we'll use to determine default parameters.

In [1]:
%pylab inline
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
import pandas as pd
import pysumma as ps

veg_igbp = pd.read_csv('./VEGPARM_IGBP_MODIS_NOAH.TBL', 
                       index_col=-1, skipinitialspace=True)
veg_igbp.index = veg_igbp.index.map(lambda x: x.strip().replace("'", ""))

soil_rosetta = pd.read_csv('./SOILPARM_ROSETTA.TBL', 
                           index_col=-1, skipinitialspace=True)
soil_rosetta.index = soil_rosetta.index.map(lambda x: x.strip().replace("'", ""))

Populating the interactive namespace from numpy and matplotlib


## Definition of the `ps.Ostrich` object

The `ps.Ostrich` object requires at least 3 inputs, along with optional specification of the python executable to use.
The required arguments are:
 * `ostrich_exe` : the path to the Ostrich executable
 * `summa_exe` : the path to the SUMMA executable
 * `file_manger` : the path to the SUMMA setup file manager to calibrate
 * `python_path` : (optional) the path to the python executable that will be used to run SUMMA via pysumma.
 
The `python_path` may be necessary for you to specify because of conda/virtual environments.
As usual, we define the object by instantiating it with these arguments.

In [2]:
site = 'BE-Bra'
summa_exe = '/pool0/data/andrbenn/summa/bin/summa.exe'
ostrich_exe = '/pool0/home/andrbenn/data/naoki_calib_example/ostrich.exe'
python_exe = '/pool0/data/andrbenn/.conda/all/bin/python'
ostrich = ps.Ostrich(ostrich_exe, summa_exe, f'./{site}/file_manager.txt', python_path=python_exe)

## Configuration of the `ps.Ostrich` object

Before the calibration can be started we have to fill out a bit more information.
The method of doing so right now is to manually assign each of the required pieces of information.
Hopefully in the future we can streamline this process while allowing for ease of customization, but for now consider the interface to be in flux.
The first thing that needs to be set is some information about how to calculate the objective function at each calibration step.
We do this with several pieces of information, first setting the observation data file and the calibration variables in both the simulated and observed datasets.
Then, you can specify a simple function to convert the observed and simulated data to match.
In our case the observed data simply needs to be multiplied by negative one.
Functions of this sort (lambdas) are the only supported method of conversion for the moment, and should be self contained (AKA not making any external function calls.
Finally, we say that we want to calibrate using root mean square error as our objective function and that we want to minimize it (or rather, that we don't want to maximize it).
As said earlier, the `cost_function` can be set to one of `RMSE`, `MAE`, or `KGE` for the moment.
If setting the `cost_function` to `KGE` then you probably want to set `maximize` to `True`.

In [3]:
ostrich.obs_data_file = f'/pool0/data/andrbenn/workspace/ostrich_example/{site}/forcings/{site}.nc'
ostrich.sim_calib_var = 'scalarLatHeatTotal'
ostrich.obs_calib_var = 'Qle_cor'
ostrich.import_strings = 'import numpy as np'
ostrich.conversion_function = lambda x: -x
ostrich.filter_function = lambda x: (x.isel(time=np.logical_and(x.time.dt.hour > 6, x.time.dt.hour < 20))
                                      .sel(time=slice('2009-04-01', '2010-09-30')))
ostrich.cost_function = 'KGE'
ostrich.maximize = True

Now, we set a couple of the optimization algorithm parameters and interact with the `ostrich` object's internal `ps.Simulation` object.
First, we will set the `max_iters` and `perturb_val` - these control how many calibration runs to do and how "hard" to perturb our test parameters (which will be defined later).
Then we set the run times of `ostrich.simulation` , which is simply a pysumma `Simulation` object wrapped inside of the `Ostrich` object.
We will also retrieve the `local_attributes` from the `ostrich.simulation` object, which contains the definitions of the soil and vegetation types which we will use to define our starting parameters.

In [4]:
ostrich.max_iters = 100
ostrich.perturb_val = 0.2
ostrich.simulation.decisions['simulStart'] = '2009-01-01 23:30'
ostrich.simulation.decisions['simulFinsh'] = '2010-10-01 23:30'
attr = ostrich.simulation.local_attributes

## Setting parameters and ranges

Once we have all of that we can work on setting up our default parameters from the soil and vegetation types.
To do so, we will select out the soil and vegetation parameters for our specific run from the soil and vegetation tables loaded in at the beginning of this notebook.
Then, I define some rooting depths that were not included in those tables for use later.
Following that, I set some `initial_values` that will be used in the following cell when creating the calibration parameter objects.

In [5]:
soil_params = soil_rosetta[soil_rosetta['SOILTYPINDEX'] == attr['soilTypeIndex'].values[0]]
veg_params = veg_igbp[veg_igbp['VEGTYPINDEX'] == attr['vegTypeIndex'].values[0]]

# Source: Zeng 2001 AMS
igbp_rooting_depths = {1: 1.8,  2: 3.0,  3: 2.0,   4: 2.0,  5: 2.4,  6: 2.5,  7: 3.10,  8: 1.7,
                       9: 2.4, 10: 1.5, 11: 0.01, 12: 1.5, 13: 1.5, 14: 1.5, 15: 0.01, 16: 4.0}

initial_values = {
    'rootingDepth': igbp_rooting_depths[attr['vegTypeIndex'].values[0]],
    'theta_res': soil_params['theta_res'].values[0],
    'theta_sat': soil_params['theta_sat'].values[0],
    'vGn_n': soil_params['vGn_n'].values[0],
    'critSoilTranspire': soil_params['REFSMC'].values[0],
    'k_soil': soil_params['k_soil'].values[0],
}

<br>
The cell following this text contains the first definition of basic calibration parameters, which are defined by `OstrichParam` objects.
These are quite simple objects which are mainly used for converting values to the correct string template that will be used by `Ostrich` internally.
The way that you construct a single `OstrichParam` is to supply the parameter name as SUMMA sees it, the starting value and a range of values that Ostrich will search within.
We provide a list of these `OstrichParam`s to the `ostrich` object in the `calib_params` attribute.
<br>

In [6]:
ostrich.calib_params = [
    ps.OstrichParam('k_soil', initial_values['k_soil'], (1e-7, 0.001)),
    ps.OstrichParam('rootingDepth', initial_values['rootingDepth'], (0.01,  5.0)),
    ps.OstrichParam('theta_res', initial_values['theta_res'],   (0.001,  0.3)),
    ps.OstrichParam('theta_sat', initial_values['theta_sat'],   (0.3,   0.85)),
    ps.OstrichParam('windReductionParam', 0.50,   (0.0,   1.0)),
    ps.OstrichParam('leafDimension', 0.04, (0.01, 0.1)),
]

<br>
Of course, this basic parameter search can lead up putting you into parameter space which is not physically possible, so we need to have the ability to "tie" parameters together with some constraints.
To do so, you can use the `add_tied_param` method on the `ostrich` object.
Currently this only provides linear, bounded support between two other parameters, and no linked-tied parameters (that is a tied parameter can't be used as a bound on another tied parameter).
The syntax here is to provide the tied parameter name, along with the parameter names that the tied parameter must be between.
Ostrich will handle the rest from here.
<br>

In [7]:
ostrich.add_tied_param('fieldCapacity', lower_bound='theta_res', upper_bound='theta_sat')
ostrich.add_tied_param('critSoilTranspire', lower_bound='theta_res', upper_bound='theta_sat')

## Running and viewing output
Finally, we can write the configuration and run the optimization routine.
For the moment I am leaving the write and run sections separate because the API is not stable and it is often useful to just write the configurationa and inspect it manually.
At some point it will likely become incorporated directly into the run method as well, but currently this is the approach we take.
Neither method requires any arguments.
The run of this for 10 two year runs (as we specifed several code cells earlier) will take a bit to run.
Following that we will simply print the standard output.
It shows the output that you would normally see during runtime of Ostrich, but more statistics and diagnostics of the calibration will be incorporated into future versions of pysumma.

In [8]:
ostrich.write_config()

In [9]:
ostrich.run()
print(ostrich.stdout)

Starting up MPI required 0.000169 seconds
--------------------------------------------------------------------------
 OSTRICH version 17.12.19 (Built Dec 19 2017 @ 12:26:43)

 A computer program for model-independent calibration and optimization.

 Author             L. Shawn Matott
 Copyright (C) 2007 L. Shawn Matott

 This program is free software; you can redistribute 
 it and/or modify it under the terms of the GNU  
 General Public License as published by the Free 
 Software Foundation; either version 2 of the 
 License, or(at your option) any later version. 

 This program is distributed in the hope that it will 
 be useful, but WITHOUT ANY WARRANTY; without even 
 the implied warranty of MERCHANTABILITY or FITNESS 
 FOR A PARTICULAR PURPOSE. See the GNU General Public 
 License for more details. 

 You should have received a copy of the GNU General 
 Public License along with this program; if not, 
 write to the Free Software Foundation, Inc., 59 
 Temple Place, Suite 330, Bosto